# Module Initialization
Here we will load all data and relevant modules. This will access helpful utilities aswell.

In [ ]:
import scipy.misc
from scipy import signal
from scipy.fftpack import fft, fftshift, ifft
from scipy.fftpack import fftfreq
import numpy as np 
import matplotlib.pyplot as plt
from utils.utils import *
from utils.preprocess import *

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from utils.utils import *
from utils.preprocess import *
from nn.rnn import *
from nn.cnn import *
from nn.cnn_lstm import *
from nn.cnn_gru import *
from nn.solver import *
import torch, torchaudio, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
import copy
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
#for macOS
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

X_test_cpy =copy.deepcopy( X_test)
y_test_cpy =copy.deepcopy( y_test)
person_train_valid_cpy =copy.deepcopy( person_train_valid)
X_train_valid_cpy =copy.deepcopy( X_train_valid)
y_train_valid_cpy =copy.deepcopy( y_train_valid)
person_test_cpy =copy.deepcopy( person_test)

static_X_test = np.load("X_test.npy")
static_y_test = np.load("y_test.npy")
static_y_test -= 769
static_X_train_valid = np.load("X_train_valid.npy")
static_y_train_valid = np.load("y_train_valid.npy") 
static_y_train_valid -= 769


In [ ]:
# If data in ./data/project
X_test = np.load(add_path("X_test.npy"))
y_test = np.load(add_path("y_test.npy"))
person_train_valid = np.load(add_path("person_train_valid.npy"))
X_train_valid = np.load(add_path("X_train_valid.npy"))
y_train_valid = np.load(add_path("y_train_valid.npy"))
person_test = np.load(add_path("person_test.npy"))
X_test_cpy =copy.deepcopy( X_test)
y_test_cpy =copy.deepcopy( y_test)
person_train_valid_cpy =copy.deepcopy( person_train_valid)
X_train_valid_cpy =copy.deepcopy( X_train_valid)
y_train_valid_cpy =copy.deepcopy( y_train_valid)
person_test_cpy =copy.deepcopy( person_test)

Check the shape of our data

In [ ]:
print('Training/Valid data shape: {}'.format(X_train_valid.shape))
print('Test data shape: {}'.format(X_test.shape))
print('Training/Valid Target shape: {}'.format(y_train_valid.shape))
print('Test Target shape: {}'.format(y_test.shape))
print('Person Train/Valid shape: {}'.format(person_train_valid.shape))
print('Person Test shape: {}'.format(person_test.shape))


# Data Preprocessing

Here we will perfrom some data preprocessing to create a larger dataset and improve the generalization of our network.

## Outline
**Describe the preprocessing that will be done**

In [ ]:
# THIS CELL ALONE WILL DO ALL PREPROCESSING
# Do all preprocessing and pair patient labels
person_test,x_test,y_test,person_valid,x_valid,y_valid,person_train, x_train, y_train = std_preprocess_EEG(
    X_test = X_test,
    y_test = y_test,
    person_train_valid = person_train_valid,
    X_train_valid = X_train_valid,
    y_train_valid = y_train_valid,
    person_test = person_test,
    val_size = 5000)
print('Training data shape: {}'.format(x_train.shape))
print('Validation data shape: {}'.format(x_valid.shape))
print('Test data shape: {}'.format(x_test.shape))
print('Training Target shape: {}'.format(y_train.shape))
print('Validation Target shape: {}'.format(y_valid.shape))
print('Test Target shape: {}'.format(y_test.shape))
print('Person Train shape: {}'.format(person_train.shape))
print('Person Validation shape: {}'.format(person_valid.shape))
print('Person Test shape: {}'.format(person_test.shape))

In [ ]:
#y_train_valid -= 769
#y_test -= 769

## Visualizing the data

ch_data = X_train_valid[:,8,:]


class_0_ind = np.where(y_train_valid == 0)
ch_data_class_0 = ch_data[class_0_ind]
avg_ch_data_class_0 = np.mean(ch_data_class_0,axis=0)


class_1_ind = np.where(y_train_valid == 1)
ch_data_class_1 = ch_data[class_1_ind]
avg_ch_data_class_1 = np.mean(ch_data_class_1,axis=0)

class_2_ind = np.where(y_train_valid == 2)
ch_data_class_2 = ch_data[class_2_ind]
avg_ch_data_class_2 = np.mean(ch_data_class_2,axis=0)

class_3_ind = np.where(y_train_valid == 3)
ch_data_class_3 = ch_data[class_3_ind]
avg_ch_data_class_3 = np.mean(ch_data_class_3,axis=0)


plt.plot(np.arange(1000),avg_ch_data_class_0)
plt.plot(np.arange(1000),avg_ch_data_class_1)
plt.plot(np.arange(1000),avg_ch_data_class_2)
plt.plot(np.arange(1000),avg_ch_data_class_3)
plt.axvline(x=500, label='line at t=500',c='cyan')

plt.legend(["Cue Onset left", "Cue Onset right", "Cue onset foot", "Cue onset tongue"])

In [ ]:
def data_filter(X_train_valid,X_test):
    num_fft=500
    X_train_valid_denoise=np.zeros(X_train_valid.shape)
    X_test_denoise=np.zeros(X_test.shape)
#define filter
    filter_denoise=scipy.signal.firwin(1001,[0.4,0.48],pass_zero='bandstop')
    for i in range(X_train_valid.shape[1]):
        for j in range(X_train_valid.shape[0]):
            X_train_valid_denoise[j,i,:]=scipy.signal.convolve(X_train_valid[j,i,:],filter_denoise,mode='same')
    for i in range(X_test.shape[1]):
        for j in range(X_test.shape[0]):
            X_test_denoise[j,i,:]=scipy.signal.convolve(X_test[j,i,:],filter_denoise,mode='same')
    return X_train_valid_denoise,X_test_denoise

In [ ]:
X_train_valid_denoise,X_test_denoise = data_filter(X_train_valid_cpy, X_test_cpy)
print('Training/Valid data shape: {}'.format(X_train_valid_denoise.shape))
print('Test data shape: {}'.format(X_test_denoise.shape))

In [ ]:
# Do all preprocessing and pair patient labels
person_test_den,x_test_den,y_test_den,person_valid_den,x_valid_den,y_valid_den,person_train_den, x_train_den, y_train_den= std_preprocess_EEG(
    X_test = X_test_denoise,
    y_test = y_test_cpy,
    person_train_valid = person_train_valid_cpy,
    X_train_valid = X_train_valid_denoise,
    y_train_valid = y_train_valid_cpy,
    person_test = person_test_cpy,
    val_size = 5000)
print('Denoised Training data shape: {}'.format(x_train_den.shape))
print('Denoised Validation data shape: {}'.format(x_valid_den.shape))
print('Denoised Test data shape: {}'.format(x_test_den.shape))
print('Denoised Training Target shape: {}'.format(y_train_den.shape))
print('Denoised Validation Target shape: {}'.format(y_valid_den.shape))
print('Denoised Test Target shape: {}'.format(y_test_den.shape))
print('Denoised Person Train shape: {}'.format(person_train_den.shape))
print('Denoised Person Validation shape: {}'.format(person_valid_den.shape))
print('Denoised Person Test shape: {}'.format(person_test_den.shape))

In [ ]:
X_train_valid_prep,y_train_valid_prep = data_prep(X_train_valid,y_train_valid,2,2,True, 0, 500)

In [ ]:
# to_categorical is a tensorflow command so here it is in python
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [ ]:
#Possible augmentations
# Create Data set with gaussian noise
std_data = np.std(X_train_valid)
X_train_valid_gn = np.random.randn(*X_train_valid.shape)*std_data/10 +X_train_valid

# Create data set with single sample shift
X_train_valid_delay = np.roll(X_train_valid,1,axis=2)

# Create data set that is scaled 
X_train_valid_half = X_train_valid/2

# Hyperparameter Tuning

In this section, we tuned our hyperparameters for each network to achieve best performance. We have commented out the hyperparameters that were tested out for each network.

## CNN


In [ ]:
num_epochs = 50 #epochs tested: 3, 5, 10, 20, 30, 50, 60
learning_rate = 0.001 #lrs tested: 0.01, 0.001, 0.0001,0.00001

# hyperparameters for LSTM
input_size = 250 #number of features 
hidden_size = 128 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers 

# class and batch size
num_classes = 4 #number of output classes 
batch_size = 64 

In [ ]:
# Create Torch Iterables
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
valid_dataset = torch.utils.data.TensorDataset(x_valid, y_valid)
test_dataset  = torch.utils.data.TensorDataset(x_test, y_test)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


In [ ]:
cnn = CNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9) #lr decays tested: 0.9, 0.95, 0.99

In [ ]:
solver = Solver(num_epochs, cnn, optimizer, scheduler, criterion, verbose=True )
solver.train(trainloader, validloader)
solver.test(testloader)

In [ ]:
plt.subplot(3, 1, 1)
plt.title('Training and validation loss for CNN')
plt.xlabel('Epoch')
plt.subplot(3, 1, 1)
plt.plot(solver.loss_history, label='training')
plt.plot(solver.val_loss_history, label='validation')
plt.legend(["train", "val"])

## LSTM

In [ ]:
num_epochs = 50 #epochs tested: 3, 5, 10, 20, 30, 50, 60
learning_rate = 0.0001 #lrs tested: 0.01, 0.001, 0.0001,0.00001

# hyperparameters for LSTM
input_size = 250 #number of features tested: 250, 1000
hidden_size = 128 #number of features in hidden state tested: 16,32,64,128,250
num_layers = 1 #number of stacked lstm layers tested: 1,2,4,6

# class and batch size
num_classes = 4 #number of output classes 
batch_size = 64 

In [ ]:
lstm = RNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9) #lr decays tested: 0.9, 0.95, 0.99


In [ ]:
solver = Solver(num_epochs, lstm, optimizer, scheduler, criterion, verbose=True,cnn_reshape=False)
solver.train(trainloader, validloader)
solver.test(testloader)

In [ ]:
plt.subplot(3, 1, 1)
plt.title('Training and validation loss for LSTM')
plt.xlabel('Epoch')
plt.subplot(3, 1, 1)
plt.plot(solver.loss_history, label='training')
plt.plot(solver.val_loss_history, label='validation')
plt.legend(["train", "val"])


## CNN_LSTM

In [ ]:
num_epochs = 50 #epochs tested: 3, 5, 10, 20, 30, 50, 60
learning_rate = 0.001 #lrs tested: 0.01, 0.001, 0.0001,0.00001

# hyperparameters for LSTM
input_size = 250 #number of features tested: 250, 1000
hidden_size = 128 #number of features in hidden state tested: 16,32,64,128,250
num_layers = 1 #number of stacked lstm layers tested: 1,2,4,6

# class and batch size
num_classes = 4 #number of output classes 
batch_size = 64 

In [ ]:
cnn_lstm = CNN_LSTM(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9) #lr decays tested: 0.9, 0.95, 0.99

In [ ]:
solver = Solver(num_epochs, cnn_lstm, optimizer, scheduler, criterion, verbose=True)
solver.train(trainloader, validloader)
solver.test(testloader)

In [ ]:
plt.subplot(3, 1, 1)
plt.title('Training and validation loss for CNN-LSTM')
plt.xlabel('Epoch')
plt.subplot(3, 1, 1)
plt.plot(solver.loss_history, label='training')
plt.plot(solver.val_loss_history, label='validation')
plt.legend(["train", "val"])

## Single Subject Classification

For this objective we are looking at how a model performs when trained on the dataset of each subject 

We test each model (RNN LSTM, CNN, CNN + LSTM)

In [ ]:
# Set new parameters here for single subject classification
num_epochs = 50 #epochs 
learning_rate = 0.001 #lr

input_size = 250 #number of features 
hidden_size = 128 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers 

num_classes = 4 #number of output classes 
batch_size = 64 

### Single Subject Classification with CNN

In [ ]:
# Train CNN NETWORK on each subject and see how the model for each subject 
# with test dataset from each subject
totalsub = 9
subject_validaccs = []
subject_testaccs = []
for sub in range(0, totalsub):
    train_dataset = torch.utils.data.TensorDataset(x_train_den[np.where(person_train_den == sub)[0]], y_train_den[np.where(person_train_den == sub)[0]])
    valid_dataset = torch.utils.data.TensorDataset(x_valid_den[np.where(person_valid_den == sub)[0]], y_valid_den[np.where(person_valid_den == sub)[0]])
    test_dataset  = torch.utils.data.TensorDataset(x_test_den[np.where(person_test_den == sub)[0]], y_test_den[np.where(person_test_den == sub)[0]])
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
    model = CNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
    criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    solver = Solver(num_epochs, model, optimizer, scheduler, criterion, verbose=True )
    solver.train(trainloader, validloader)
    subject_validaccs.append(solver.val_accuracy_history[-1])
    print("subject " + str(sub+1) + " validation accuracy: %1.5f"% (solver.val_accuracy_history[-1]))
    solver.test(testloader)
    subject_testaccs.append(solver.test_accuracy)
    print("subject " + str(sub+1) + " test accuracy: %1.5f"% (solver.test_accuracy))
    
    plt.subplot(3, 1, 1)
    plt.title('Training and validation loss per subject')
    plt.xlabel('Epoch')
    plt.subplot(3, 1, 1)
    plt.plot(solver.loss_history, label='training')
    plt.plot(solver.val_loss_history, label='validation')
    plt.legend(["train", "val"])


In [ ]:
plt.subplot(3, 1, 1)
plt.title('Final validation and test accuracy per subject using CNN model')
plt.ylabel('Model accuracy')
plt.subplot(3, 1, 1)
plt.plot(subject_validaccs, label='val')
plt.plot(subject_testaccs, label='test')
plt.legend(["val", "test"])
plt.subplot(3, 1, 2)
subjectnum = range(0, totalsub)
plt.scatter(subjectnum, subject_validaccs)
plt.scatter(subjectnum, subject_testaccs)
plt.xlabel('Subject')
plt.ylabel('Model accuracy')
plt.legend(["val", "test"])

cnn_subject_validaccs = subject_validaccs.copy()
cnn_subject_testaccs = subject_testaccs.copy()

### Single Subject Classification with LSTM

In [ ]:
# Change to optimal learning rate of 0.0001 for LSTM
learning_rate = 0.0001 #lr

# Train RNN LSTM NETWORK on each subject and see how the model for each subject 
# with test dataset from each subject
totalsub = 9
subject_validaccs = []
subject_testaccs = []
for sub in range(0, totalsub):
    train_dataset = torch.utils.data.TensorDataset(x_train_den[np.where(person_train_den == sub)[0]], y_train_den[np.where(person_train_den == sub)[0]])
    valid_dataset = torch.utils.data.TensorDataset(x_valid_den[np.where(person_valid_den == sub)[0]], y_valid_den[np.where(person_valid_den == sub)[0]])
    test_dataset  = torch.utils.data.TensorDataset(x_test_den[np.where(person_test_den == sub)[0]], y_test_den[np.where(person_test_den == sub)[0]])
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
    model = RNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
    criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    solver = Solver(num_epochs, model, optimizer, scheduler, criterion, verbose=True, cnn_reshape=False)
    solver.train(trainloader, validloader)
    subject_validaccs.append(solver.val_accuracy_history[-1])
    print("subject " + str(sub+1) + " validation accuracy: %1.5f"% (solver.val_accuracy_history[-1]))
    solver.test(testloader)
    subject_testaccs.append(solver.test_accuracy)
    print("subject " + str(sub+1) + " test accuracy: %1.5f"% (solver.test_accuracy))
    
    plt.subplot(3, 1, 1)
    plt.title('Training and validation loss per subject')
    plt.xlabel('Epoch')
    plt.subplot(3, 1, 1)
    plt.plot(solver.loss_history, label='training')
    plt.plot(solver.val_loss_history, label='validation')
    plt.legend(["train", "val"])

In [ ]:
plt.subplot(3, 1, 1)
plt.title('Final validation and test accuracy per subject using LSTM model')
plt.ylabel('Model accuracy')
plt.subplot(3, 1, 1)
plt.plot(subject_validaccs, label='val')
plt.plot(subject_testaccs, label='test')
plt.legend(["val", "test"])
plt.subplot(3, 1, 2)
subjectnum = range(0, totalsub)
plt.scatter(subjectnum, subject_validaccs)
plt.scatter(subjectnum, subject_testaccs)
plt.xlabel('Subject')
plt.ylabel('Model accuracy')
plt.legend(["val", "test"])


lstm_subject_validaccs = subject_validaccs.copy()
lstm_subject_testaccs = subject_testaccs.copy()

### Single Subject Classification with CNN-LSTM

In [ ]:
# Change to optimal learning rate for CNN-LSTM
learning_rate = 0.001 #lr

# Train CNN + LSTM NETWORK on each subject and see how the model for each subject 
# with test dataset from each subject
totalsub = 9
subject_validaccs = []
subject_testaccs = []
for sub in range(0, totalsub):
    train_dataset = torch.utils.data.TensorDataset(x_train_den[np.where(person_train_den == sub)[0]], y_train_den[np.where(person_train_den == sub)[0]])
    valid_dataset = torch.utils.data.TensorDataset(x_valid_den[np.where(person_valid_den == sub)[0]], y_valid_den[np.where(person_valid_den == sub)[0]])
    test_dataset  = torch.utils.data.TensorDataset(x_test_den[np.where(person_test_den == sub)[0]], y_test_den[np.where(person_test_den == sub)[0]])
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
    model = CNN_LSTM(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
    criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    solver = Solver(num_epochs, model, optimizer, scheduler, criterion, verbose=True)
    solver.train(trainloader, validloader)
    subject_validaccs.append(solver.val_accuracy_history[-1])
    print("subject " + str(sub+1) + " validation accuracy: %1.5f"% (solver.val_accuracy_history[-1]))
    solver.test(testloader)
    subject_testaccs.append(solver.test_accuracy)
    print("subject " + str(sub+1) + " test accuracy: %1.5f"% (solver.test_accuracy))
    
    plt.subplot(3, 1, 1)
    plt.title('Training and validation loss per subject')
    plt.xlabel('Epoch')
    plt.subplot(3, 1, 1)
    plt.plot(solver.loss_history, label='training')
    plt.plot(solver.val_loss_history, label='validation')
    plt.legend(["train", "val"])

In [ ]:
plt.subplot(3, 1, 1)
plt.title('Final validation and test accuracy per subject using CNN + LSTM model')
plt.ylabel('Model accuracy')
plt.subplot(3, 1, 1)
plt.plot(subject_validaccs, label='val')
plt.plot(subject_testaccs, label='test')
plt.legend(["val", "test"])
plt.subplot(3, 1, 2)
subjectnum = range(0, totalsub)
plt.scatter(subjectnum, subject_validaccs)
plt.scatter(subjectnum, subject_testaccs)
plt.xlabel('Subject')
plt.ylabel('Model accuracy')
plt.legend(["val", "test"])

cnnlstm_subject_validaccs = subject_validaccs.copy()
cnnlstm_subject_testaccs = subject_testaccs.copy()

In [ ]:
print(cnn_subject_validaccs)
print(cnn_subject_testaccs)

In [ ]:
print(lstm_subject_validaccs)
print(lstm_subject_testaccs)

In [ ]:
print(cnnlstm_subject_validaccs)
print(cnnlstm_subject_testaccs)

If we look at the classification accuracy of a single subject, in this case subject 1 (ID 0), the CNN model achieves the best validation and test accuracy.

## Classification Accuracy as function of time (III)

In [ ]:
# define function that cuts data to specific time size but keeps same preprocessing
    
def preprocess_data(X_train_valid, y_train_valid, X_test,y_test, sub_sample, average,noise, trim_begin, trim_end, batch_size):
    X_train_valid_prep,y_train_valid_prep = data_prep(X_train_valid,y_train_valid,sub_sample,average, noise, trim_begin, trim_end)
    x_test, y_test = data_prep(X_test,y_test,sub_sample,average, noise, trim_begin, trim_end)

    ## Random splitting and reshaping the data

    # First generating the training and validation indices using random splitting
    ind_valid = np.random.choice(8460, 5000, replace=False)
    ind_train = np.array(list(set(range(8460)).difference(set(ind_valid))))

    # Creating the training and validation sets using the generated indices
    (x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid] 
    (y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]

    # change to tensor
    x_train = Variable(torch.Tensor(x_train))
    x_valid = Variable(torch.Tensor(x_valid))
    x_test = Variable(torch.Tensor(x_test))

    y_train = Variable(torch.Tensor(y_train))
    y_train = torch.reshape(y_train,  (y_train.shape[0], 1)) 

    y_valid = Variable(torch.Tensor(y_valid))
    y_valid = torch.reshape(y_valid,  (y_valid.shape[0], 1)) 

    y_test = Variable(torch.Tensor(y_test))
    y_test = torch.reshape(y_test, (y_test.shape[0], 1))
    print("")
    print('final shape')
    print("Training Shape", x_train.shape, y_train.shape)
    print("Valid Shape", x_valid.shape, y_valid.shape)
    print("Test Shape", x_test.shape, y_test.shape)
    
    train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
    valid_dataset = torch.utils.data.TensorDataset(x_valid, y_valid)
    test_dataset  = torch.utils.data.TensorDataset(x_test, y_test)
    
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
    return trainloader, validloader, testloader, x_train
    
testing = preprocess_data(static_X_train_valid, static_y_train_valid, static_X_test,static_y_test, 2, 2,True, 0, 500, 64)

In [ ]:
num_epochs = 20 #epochs 
learning_rate = 0.001 #0.001 lr

input_size = 250 #number of features 
hidden_size = 128 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers 

num_classes = 4 #number of output classes 
batch_size = 64 

In [ ]:
timesteps_test = [100,200,300,400,500,600,700,800,900,1000]
val_accuracies = {}
test_accuracies = {}
losses = []

for timestep in timesteps_test:
    trainloader, validloader, testloader, x_train = preprocess_data(static_X_train_valid, static_y_train_valid, static_X_test,static_y_test, 2, 2,True, 0, timestep, 64)
    #define new model to fit data
    lstm1 = RNN(num_classes, timestep//2, hidden_size, num_layers, x_train.shape[1])
    criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
    optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    highest_val = 0
    loss_array = []
    for epoch in range(num_epochs):
      for i, data in enumerate(trainloader,0):
        inputs, labels = data
        outputs = lstm1.forward(inputs) #forward pass
        optimizer.zero_grad() #calculate the gradient, manually setting to 0
        loss = criterion(outputs, labels.reshape(labels.size(0),).type(torch.long))
        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
        
      scheduler.step()
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
      loss_array = np.append(loss_array, loss.item())  
      correct = 0
      total = 0
      with torch.no_grad():
        for data in validloader:
            inputs, labels = data
            outputs = lstm1(inputs)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.reshape(labels.size(0),)).sum().item()
      print("  Val Accuracy: %1.5f"% (float(correct) / float(total)))
      if (float(correct) / float(total)) > highest_val:
          highest_val = (float(correct) / float(total))
    val_accuracies[timestep] = highest_val
    #test set
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            outputs = lstm1(inputs)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.reshape(labels.size(0),)).sum().item()
        print("Test Accuracy: %1.5f"% (float(correct) / float(total)))
    test_accuracies[timestep] = (float(correct) / float(total))
    losses += [loss_array]

In [ ]:
print("Validation accuracies as function of time", val_accuracies)
print("")
print("Test accuracies as function of time", test_accuracies)

In [ ]:
plt.subplot(2,1,1)
plt.plot(val_accuracies.values())
plt.plot(test_accuracies.values())
plt.xlabel('Time x 100 cutoff')
plt.ylabel('Model accuracy')
plt.legend(["val_acc", "test_acc"])
plt.subplot(2,1,2)
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss Curves')
plt.legend(["100", "200", "300", "400", "500", "600", "700", "800", "900", "1000"])
# plt.plot(losses)

0-700 timesteps had the highest classification accuracy on test data and 0-1000 timesteps had highest classification accuracy on validation data

# Optimizing Classification across all subjects

In this section we will be tuning the hyperparameters of our nerual nets across all subjects. We will examine any interesting trends.

In [ ]:
num_epochs = 50 #epochs 
learning_rate = 0.001 #0.0001 lr for LSTM only

input_size = 250 #number of features 
hidden_size = 128 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers 

num_classes = 4 #number of output classes 
batch_size = 64 

In [ ]:
# Create Torch Itterables
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
valid_dataset = torch.utils.data.TensorDataset(x_valid, y_valid)
test_dataset  = torch.utils.data.TensorDataset(x_test, y_test)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


In [ ]:
# Create Filtered Torch Itterables
filtered_train_dataset = torch.utils.data.TensorDataset(x_train_den, y_train_den)
filtered_valid_dataset = torch.utils.data.TensorDataset(x_valid_den, y_valid_den)
filtered_test_dataset  = torch.utils.data.TensorDataset(x_test_den, y_test_den)
filtered_trainloader = torch.utils.data.DataLoader(filtered_train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
filtered_validloader = torch.utils.data.DataLoader(filtered_valid_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
filtered_testloader = torch.utils.data.DataLoader(filtered_test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
# Create CNN, classifier, and optimizer
cnn_only = CNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
cnn_criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
cnn_optimizer = torch.optim.Adam(cnn_only.parameters(), lr=learning_rate)
cnn_scheduler = torch.optim.lr_scheduler.ExponentialLR(cnn_optimizer, gamma=0.9)

# Create CNN, classifier, and optimizer for filtered data comparrison
cnn_only_filt = CNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
cnn_filt_criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
cnn_filt_optimizer = torch.optim.Adam(cnn_only_filt.parameters(), lr=learning_rate)
cnn_filt_scheduler = torch.optim.lr_scheduler.ExponentialLR(cnn_filt_optimizer, gamma=0.9)

# Create RNN, classifier, and optimizer
rnn_only = RNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
rnn_criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
rnn_optimizer = torch.optim.Adam(rnn_only.parameters(), lr=learning_rate)
rnn_scheduler = torch.optim.lr_scheduler.ExponentialLR(rnn_optimizer, gamma=0.9)

# Create RNN, classifier, and optimizer for filtered data comparrison
rnn_only_filt = RNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
rnn_filt_criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
rnn_filt_optimizer = torch.optim.Adam(rnn_only_filt.parameters(), lr=learning_rate)
rnn_filt_scheduler = torch.optim.lr_scheduler.ExponentialLR(rnn_filt_optimizer, gamma=0.9)

# Create CNN-LSTM, classifier, and optimizer
cnn_lstm = CNN_LSTM(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
cnn_lstm_criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
cnn_lstm_optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr=learning_rate)
cnn_lstm_scheduler = torch.optim.lr_scheduler.ExponentialLR(cnn_lstm_optimizer, gamma=0.9)

# Create CNN-LSTM, classifier, and optimizer for filtered data comparrison
cnn_lstm_filt = CNN_LSTM(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
cnn_lstm_filt_criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
cnn_lstm_filt_optimizer = torch.optim.Adam(cnn_lstm_filt.parameters(), lr=learning_rate)
cnn_lstm_filt_scheduler = torch.optim.lr_scheduler.ExponentialLR(cnn_lstm_filt_optimizer, gamma=0.9)

In [ ]:
# Train CNN 
cnn_solver = Solver(num_epochs, cnn_only,cnn_optimizer, cnn_scheduler, cnn_criterion, verbose=False )
cnn_solver.train(trainloader, validloader)
print("CNN Training Done!")

# Train RNN
rnn_solver = Solver(num_epochs, rnn_only,rnn_optimizer, rnn_scheduler, rnn_criterion, verbose=False, cnn_reshape=False )
rnn_solver.train(trainloader, validloader)
print("RNN Training Done!")

# Train CNN-LSTM
cnn_lstm_solver = Solver(num_epochs, cnn_lstm,cnn_lstm_optimizer, cnn_lstm_scheduler, cnn_lstm_criterion, verbose=False )
cnn_lstm_solver.train(trainloader, validloader)
print("CNN-LSTM Training Done!")

# Train CNN on filtered Data
cnn_filt_solver = Solver(num_epochs, cnn_only_filt,cnn_filt_optimizer, cnn_filt_scheduler, cnn_filt_criterion, verbose=False )
cnn_filt_solver.train(filtered_trainloader, filtered_validloader)
print("CNN Training on Filtered Data Done!")

# Train RNN on filtered Data
rnn_filt_solver = Solver(num_epochs, rnn_only_filt,rnn_filt_optimizer, rnn_filt_scheduler, rnn_filt_criterion, verbose=False, cnn_reshape=False )
rnn_filt_solver.train(filtered_trainloader, filtered_validloader)
print("RNN Training on Filtered Data Done!")

# Train CNN-LSTM on filtered Data
cnn_lstm_filt_solver = Solver(num_epochs, cnn_lstm_filt,cnn_lstm_filt_optimizer, cnn_lstm_filt_scheduler, cnn_lstm_filt_criterion, verbose=False )
cnn_lstm_filt_solver.train(filtered_trainloader, filtered_validloader)
print("CNN-LSTM Training on Filtered Data Done!")


In [ ]:
# Plot CNN Results
fig, axs = plt.subplots(2)
fig.suptitle('CNN Training Breakdown')
axs[0].plot(cnn_solver.loss_history)
axs[0].plot(cnn_solver.val_loss_history)
axs[0].plot(cnn_filt_solver.loss_history)
axs[0].plot(cnn_filt_solver.val_loss_history)
axs[0].legend(["Unfiltered train", "Unfiltered val", "Filtered Train", "Filtered Validation"])
axs[0].set_ylabel("Loss")
axs[0].set_xlabel("Epoch")
axs[0].set_title("CNN Loss Curves")
axs[1].plot(cnn_solver.val_accuracy_history)
axs[1].plot(cnn_filt_solver.val_accuracy_history)
axs[1].legend(["Unfiltered Validation Accuracy", "Filtered Validation Accuracy"])
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Epoch")
axs[1].set_title("CNN Accuracy Over Training")

In [ ]:
# Plot RNN Results
fig, axs = plt.subplots(2)
fig.suptitle('RNN Training Breakdown')
axs[0].plot(rnn_solver.loss_history)
axs[0].plot(rnn_solver.val_loss_history)
axs[0].plot(rnn_filt_solver.loss_history)
axs[0].plot(rnn_filt_solver.val_loss_history)
axs[0].legend(["Unfiltered train", "Unfiltered val", "Filtered Train", "Filtered Validation"])
axs[0].set_ylabel("Loss")
axs[0].set_xlabel("Epoch")
axs[0].set_title("RNN Loss Curves")
axs[1].plot(rnn_solver.val_accuracy_history)
axs[1].plot(rnn_filt_solver.val_accuracy_history)
axs[1].legend(["Unfiltered Validation Accuracy", "Filtered Validation Accuracy"])
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Epoch")
axs[1].set_title("RNN Accuracy Over Training")

In [ ]:
# Plot CNN-LSTM Results
fig, axs = plt.subplots(2)
fig.suptitle('CNN-LSTM Training Breakdown')
axs[0].plot(cnn_lstm_solver.loss_history)
axs[0].plot(cnn_lstm_solver.val_loss_history)
axs[0].plot(cnn_lstm_filt_solver.loss_history)
axs[0].plot(cnn_lstm_filt_solver.val_loss_history)
axs[0].legend(["Unfiltered train", "Unfiltered val", "Filtered Train", "Filtered Validation"])
axs[0].set_ylabel("Loss")
axs[0].set_xlabel("Epoch")
axs[0].set_title("CNN-LSTM Loss Curves")
axs[1].plot(cnn_lstm_solver.val_accuracy_history)
axs[1].plot(cnn_lstm_filt_solver.val_accuracy_history)
axs[1].legend(["Unfiltered Validation Accuracy", "Filtered Validation Accuracy"])
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Epoch")
axs[1].set_title("CNN-LSTM Accuracy Over Training")

In [ ]:
cnn_accuracy = cnn_solver.test(testloader)
print("CNN Test Accuracy: %1.5f"% (cnn_accuracy))
cnn_filt_accuracy = cnn_filt_solver.test(filtered_testloader)
print("CNN Test Accuracy(Filtered): %1.5f"% (cnn_filt_accuracy))
rnn_accuracy = rnn_solver.test(testloader)
print("RNN Test Accuracy: %1.5f"% (rnn_accuracy))
rnn_filt_accuracy = rnn_filt_solver.test(filtered_testloader)
print("RNN Test Accuracy(Filtered): %1.5f"% (rnn_filt_accuracy))
cnn_lstm_accuracy = cnn_lstm_solver.test(testloader)
print("CNN-LSTM Test Accuracy: %1.5f"% (cnn_lstm_accuracy))
cnn_lstm_filt_accuracy = cnn_lstm_filt_solver.test(filtered_testloader)
print("CNN-LSTM Test Accuracy(Filtered): %1.5f"% (cnn_lstm_filt_accuracy))

In [ ]:
# CNN Test Accuracy Breakdown
totalsub=9
CNN_accuracy_by_class = cnn_solver.test_by_class(testloader)
CNN_accuracy_by_patient = np.zeros(totalsub)

# Patient Accuracies
for sub in range(totalsub):
    # Partition Test set by patient number
    sub_test_dataset  = torch.utils.data.TensorDataset(x_test[np.where(person_test == sub)[0]], y_test[np.where(person_test == sub)[0]])
    sub_testloader = torch.utils.data.DataLoader(sub_test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
    CNN_accuracy_by_patient[sub] = cnn_solver.test(sub_testloader)
fig, axs = plt.subplots(2)
fig.suptitle('CNN Accuracy Breakdown')
axs[0].scatter(np.arange(num_classes),CNN_accuracy_by_class)
axs[0].set_ylabel("Accuracy")
axs[0].set_xlabel("Class")
axs[1].scatter(np.arange(totalsub),CNN_accuracy_by_patient)
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Subject Index")

In [ ]:
# RNN Test Accuracy Breakdown
totalsub=9
RNN_accuracy_by_class = rnn_solver.test_by_class(testloader)
RNN_accuracy_by_patient = np.zeros(totalsub)

# Patient Accuracies
for sub in range(totalsub):
    # Partition Test set by patient number
    sub_test_dataset  = torch.utils.data.TensorDataset(x_test[np.where(person_test == sub)[0]], y_test[np.where(person_test == sub)[0]])
    sub_testloader = torch.utils.data.DataLoader(sub_test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
    RNN_accuracy_by_patient[sub] = rnn_solver.test(sub_testloader)
fig, axs = plt.subplots(2)
fig.suptitle('LSTM Accuracy Breakdown')

axs[0].scatter(np.arange(num_classes),RNN_accuracy_by_class)
axs[0].set_ylabel("Accuracy")
axs[0].set_xlabel("Class")
axs[1].scatter(np.arange(totalsub),RNN_accuracy_by_patient)
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Subject Index")

In [ ]:
# CNN-LSTM Test Accuracy Breakdown
totalsub=9
CNN_LSTM_accuracy_by_class = cnn_lstm_solver.test_by_class(testloader)
CNN_LSTM_accuracy_by_patient = np.zeros(totalsub)

# Patient Accuracies
for sub in range(totalsub):
    # Partition Test set by patient number
    sub_test_dataset  = torch.utils.data.TensorDataset(x_test[np.where(person_test == sub)[0]], y_test[np.where(person_test == sub)[0]])
    sub_testloader = torch.utils.data.DataLoader(sub_test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
    CNN_LSTM_accuracy_by_patient[sub] = cnn_lstm_solver.test(sub_testloader)
fig, axs = plt.subplots(2)
fig.suptitle('CNN-LSTM Accuracy Breakdown')
axs[0].scatter(np.arange(num_classes),CNN_LSTM_accuracy_by_class)
axs[0].set_ylabel("Accuracy")
axs[0].set_xlabel("Class")
axs[1].scatter(np.arange(totalsub),CNN_LSTM_accuracy_by_patient)
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Subject Index")

In [ ]:
fig, axs = plt.subplots(3)
fig.suptitle('Loss Breakdown')
axs[2].plot(cnn_lstm_solver.loss_history)
axs[2].plot(cnn_lstm_solver.val_loss_history)
axs[2].plot(cnn_lstm_filt_solver.loss_history)
axs[2].plot(cnn_lstm_filt_solver.val_loss_history)
axs[2].legend(["Unfiltered train", "Unfiltered val", "Filtered Train", "Filtered Validation"])
axs[2].set_ylabel("Loss")
axs[2].set_xlabel("Epoch")
axs[2].set_title("CNN-LSTM Loss Curves")

# Plot RNN Results
axs[1].plot(rnn_solver.loss_history)
axs[1].plot(rnn_solver.val_loss_history)
axs[1].plot(rnn_filt_solver.loss_history)
axs[1].plot(rnn_filt_solver.val_loss_history)
axs[1].legend(["Unfiltered train", "Unfiltered val", "Filtered Train", "Filtered Validation"])
axs[1].set_ylabel("Loss")
axs[1].set_xlabel("Epoch")
axs[1].set_title("RNN Loss Curves")

# Plot CNN Results
axs[0].plot(cnn_solver.loss_history)
axs[0].plot(cnn_solver.val_loss_history)
axs[0].plot(cnn_filt_solver.loss_history)
axs[0].plot(cnn_filt_solver.val_loss_history)
axs[0].legend(["Unfiltered train", "Unfiltered val", "Filtered Train", "Filtered Validation"])
axs[0].set_ylabel("Loss")
axs[0].set_xlabel("Epoch")
axs[0].set_title("CNN Loss Curves")
plt.tight_layout()


In [ ]:
# Plot CNN Results
fig, axs = plt.subplots(3)
fig.suptitle('Validation Accuracy Breakdown')

axs[0].plot(cnn_solver.val_accuracy_history)
axs[0].plot(cnn_filt_solver.val_accuracy_history)
axs[0].legend(["Unfiltered Validation Accuracy", "Filtered Validation Accuracy"])
axs[0].set_ylabel("Accuracy")
axs[0].set_xlabel("Epoch")
axs[0].set_title("CNN ValidationAccuracy")

# Plot RNN Results

axs[1].plot(rnn_solver.val_accuracy_history)
axs[1].plot(rnn_filt_solver.val_accuracy_history)
axs[1].legend(["Unfiltered Validation Accuracy", "Filtered Validation Accuracy"])
axs[1].set_ylabel("Accuracy")
axs[1].set_xlabel("Epoch")
axs[1].set_title("RNN Validation Accuracy")

# Plot CNN-LSTM Results

axs[2].plot(cnn_lstm_solver.val_accuracy_history)
axs[2].plot(cnn_lstm_filt_solver.val_accuracy_history)
axs[2].legend(["Unfiltered Validation Accuracy", "Filtered Validation Accuracy"])
axs[2].set_ylabel("Accuracy")
axs[2].set_xlabel("Epoch")
axs[2].set_title("CNN-LSTM Validation Accuracy")

print("CNN Max Validation Accuracy:           %1.5f"% (np.max(cnn_solver.val_accuracy_history)))
print("CNN Max Validation Accuracy(Filtered): %1.5f"% (np.max(cnn_filt_solver.val_accuracy_history)))
print("RNN Max Validation Accuracy:           %1.5f"% (np.max(rnn_solver.val_accuracy_history)))
print("RNN Max Validation Accuracy(Filtered): %1.5f"% (np.max(rnn_filt_solver.val_accuracy_history)))
print("CNN-LSTM Max Validation Accuracy          : %1.5f"% (np.max(cnn_lstm_solver.val_accuracy_history)))
print("CNN-LSTM Max Validation Accuracy(Filtered): %1.5f"% (np.max(cnn_lstm_filt_solver.val_accuracy_history)))


# Architecture validation

In [ ]:
# Compare CNN to CNN-LSTM
cnn_accs = []
cnn_lstm_accs = []
for i in range(10):
    all_subjects_net_cnn = CNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
    all_subjects_net = CNN_LSTM(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
    criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
    optimizer = torch.optim.Adam(all_subjects_net.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    criterion1 = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
    optimizer1 = torch.optim.Adam(all_subjects_net_cnn.parameters(), lr=learning_rate)
    scheduler1 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    
    solver = Solver(num_epochs, all_subjects_net,optimizer, scheduler, criterion, verbose=True )
    solver.train(trainloader, validloader)
    solver1 = Solver(num_epochs, all_subjects_net_cnn,optimizer1, scheduler1, criterion1, verbose=True )
    solver1.train(trainloader, validloader)

    cnn_accs.append(solver1.test(testloader))
    cnn_lstm_accs.append(solver.test(testloader))

print(np.mean(cnn_accs))
print(np.mean(cnn_lstm_accs))

In [ ]:
    all_subjects_net_cnn = CNN(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
    all_subjects_net = CNN_LSTM(num_classes, input_size, hidden_size, num_layers, x_train.shape[1])
    criterion = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
    optimizer = torch.optim.Adam(all_subjects_net.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    criterion1 = torch.nn.CrossEntropyLoss()   # CE Loss is our softmax
    optimizer1 = torch.optim.Adam(all_subjects_net_cnn.parameters(), lr=learning_rate)
    scheduler1 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    
    solver = Solver(50, all_subjects_net,optimizer, scheduler, criterion, verbose=False )
    solver.train(trainloader, validloader)
    solver1 = Solver(50, all_subjects_net_cnn,optimizer1, scheduler1, criterion1, verbose=False )
    solver1.train(trainloader, validloader)

    cnn_acc = solver1.test(testloader)
    cnn_lstm_acc = solver.test(testloader)
    print("CNN Accuracy: %1.5f"% (cnn_acc))
    print("CNN_LSTM Accuracy: %1.5f"% (cnn_lstm_acc))

In [ ]:
plt.plot(solver.loss_history)
plt.plot(solver.val_loss_history)
plt.legend(["train", "val"])

In [ ]:
plt.plot(solver1.loss_history)
plt.plot(solver1.val_loss_history)
plt.legend(["train", "val"])